In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/company-bankruptcy-prediction/data.csv")

In [ ]:
df.describe()

In [ ]:
(df.isnull().sum())/len(df) * 100

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.countplot(x='Bankrupt?',data=df)

### Heavily imbalanced data. Training set will be oversampled using SMOTEENN

In [ ]:
print(df['Bankrupt?'].value_counts())

#### Let's check the correlation of features to the Bankrup label.

In [ ]:
plt.figure(figsize=(20,10))
df.corr()['Bankrupt?'][:].sort_values().plot(kind='bar')

#### There are too many features and need to filter out them. Let's use those with correlation > 0.2

In [ ]:
cor_target = abs(df.corr()['Bankrupt?'])
set_features = cor_target[cor_target > 0.2]
print(set_features)

In [ ]:
df = df[set_features.index]
df.head()

In [ ]:
plt.figure(figsize=(12,8))
df.corr()['Bankrupt?'][:].sort_values().plot(kind='bar')

In [ ]:
#Pick up feature set and the labels
X = df.drop('Bankrupt?',axis=1).values
y = df['Bankrupt?'].values

#### The dataset is skewed, so oversample only the train data with SMOTE. Keep the test data separate to prevent any feature leakage.

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from collections import Counter

#Split the dataset into 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)


#Do oversampling for the train set using SMOTE
from imblearn.combine import SMOTEENN

smn = SMOTEENN()


print("Counts before SMOTE")
print(Counter(y_train))

X_res,y_res = smn.fit_resample(X_train,y_train)
print("Counts after SMOTE")
print(Counter(y_res))



In [ ]:
classifier = XGBClassifier(objective= 'binary:logistic', eval_metric='error', max_depth = 8,max_delta_step=1,
                           learning_rate= 0.0003, n_estimators=6000,subsample=0.5,
                           use_label_encoder=False)

classifier.fit(X_res, y_res)

In [ ]:
# evaluate predictions
from sklearn.metrics import confusion_matrix, average_precision_score, roc_auc_score, roc_curve, classification_report, precision_recall_curve, f1_score
    
y_prob=classifier.predict_proba(X_test)
y_pred = classifier.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
roc_xgb = roc_auc_score(y_test, y_prob[:,1])
print('ROC-AUC', roc_xgb)
print('='*20)
print('Confusion Matrix')
cm_xgb = confusion_matrix(y_test, y_pred)
print(cm_xgb)
sns.heatmap(cm_xgb, annot=True, cmap='viridis')
cl_xgb = classification_report(y_test,y_pred )
print(cl_xgb)

## Logistic Regressions Model

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
logmodel = LogisticRegression(C=3.0, max_iter=1000)
logmodel.fit(X_res,y_res)

In [ ]:
predictions = logmodel.predict(X_test)
accuracy = metrics.accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
y_prob=logmodel.predict_proba(X_test)
#print('ROC-AUC')
roc_lin = roc_auc_score(y_test, y_prob[:,1])
print('ROC-AUC:', roc_lin)
print('='*20)
print(' Confusion Matrix')
cm_lin = confusion_matrix(y_test, predictions)
print(cm_lin)
sns.heatmap(cm_lin, annot=True, cmap='viridis')
cl_lin = classification_report(y_test,predictions)
print(cl_lin)